In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import AutoProcessor, AutoModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"

canon_df = pd.read_csv(file_path)

canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

df = canon_df[['ImageFilePath', 'ScientificName']]


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "facebook/regnet-y-040"

model = AutoModel.from_pretrained(model_name).eval().to(device)

processor = AutoProcessor.from_pretrained("facebook/regnet-y-040")


In [6]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()

In [7]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"])])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 1622/1622 [02:13<00:00, 12.15it/s]


In [ ]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(256, 128), activation='logistic', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


In [11]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax discedens,Mecyclothorax konanus,Mecyclothorax konanus
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus,Mecyclothorax konanus,Trechus obtusus,Trechus obtusus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Trechus obtusus,Trechus obtusus


In [12]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.323077,0.493634,0.323077,0.350835,0.230974,0.172371
LogisticRegression,LogisticRegression,0.590769,0.538187,0.590769,0.551428,0.148478,0.332539
SVMLinear,SVMLinear,0.603077,0.596438,0.603077,0.54448,0.150667,0.338111
SVMPolynomial,SVMPolynomial,0.467692,0.385478,0.467692,0.385965,0.11173,0.071973
SVMRadialBasis,SVMRadialBasis,0.6,0.475945,0.6,0.53079,0.135541,0.329462
NearestNeighbor,NearestNeighbor,0.498462,0.466971,0.498462,0.47466,0.156351,0.189335
RandomForest,RandomForest,0.572308,0.458003,0.572308,0.508545,0.129062,0.280166
MLP_Baseline,MLP_Baseline,0.618462,0.55814,0.618462,0.582978,0.201796,0.392482


In [13]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/35.RegNet-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/35.RegNet-linear-probing-species-metrics.csv", index=False)